In [ ]:
from IPython.display import HTML

<br><hr>

<img align="left" src="https://cgsm.org/wp-content/uploads/2019/12/berkeley-haas-wordmark_square-gold-white-on-blue.png" width="100" height="100">
<link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Merriweather">
<h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Professional Certificate Capstone 1<br>Artificial Intelligence & Machine Learning</h1>

In [ ]:
HTML('<p align="center"><div style="padding:75% 0 0 0;position:relative;"><iframe src="https://player.vimeo.com/video/739505155?autoplay=1&loop=1&h=b7141f3616&amp;badge=0&amp;autopause=0&amp;player_id=0&amp;app_id=58479" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen style="position:absolute;top:0;left:0;width:100%;height:100%;" title="Berkeley AIML CAPSTONE1"></iframe></div><script src="https://player.vimeo.com/api/player.js"></script></p>')

<img align="center" src="https://www.signitysolutions.com/blog/wp-content/uploads/2020/09/artificial-intelligence-in-healthcare-sector.jpg">

Overview: In this capstone, my goal is to do the heavy lifting of the project. I will explore my data source, test a few of the techniques, and com up with a solution or answer to my research problem. 

***Research Problem***
+ Hospitals are slammed with readmissions of patients. Specifically, data has been collected related to readmisssion as well as other outcomes pertaining to patients with diabetes. What can we do to reduce these readmissions and possibly improve patient health?

***Data Source***
+ https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008 (Links to an external site.) - This data was donated in 2014 and is a collection of the past 6 years. 

***Modeling Techniques To Be Used:***
+ Multi-class classification using one of Logistic Regression or Decision Trees, Counterfactual technique

***Expected Results***
+ I am expecting to find what are the important things (features) that contribute to a return/readmission. From this point, perhaps use the counterfactual explanations technique to determine what can possibly be changed to get a readmission score of False. 

***Why This Question Is Important***
+ This question is important because individuals are being readmitted to hospitals for the same things. Particularly, in this case, these are diabetic patients. Diabetes is morbidity that needs to be watched closely and coupled with other issues that can lead to complexity and/or fatality. Thus, perhaps we can provide recommendations on changes to 1) keep the patient healthier and 2) keep readmissions down amongst diabetic patients.

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Understanding the Data</h1><hr>

From our data source, we are presented with various data from over 10 years of clinical care at 130 US hospitals and integrated delivery networks. It includes over 40 features representing patient and hospital outcomes. Information was extracted from the database for encounters that satisfied the following criteria:
+ It is an inpatient encounter (a hospital admission).
+ It is a diabetic encounter, that is, one during which any kind of diabetes was entered to the system as a diagnosis.
+ The length of stay was at least 1 day and at most 14 days.
+ Laboratory tests were performed during the encounter.
+ Medications were administered during the encounter.

The data contains such attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab test performed, HbA1c test result, diagnosis, number of medication, diabetic medications, number of outpatient, inpatient, and emergency visits in the year before the hospitalization, etc.

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Prerequisites + Reading In The Data</h1><hr>

Here we will use pandas to read in the dataset diabetic_data.csv and assign a meaningful variable name. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import preprocessing
from tqdm import tqdm
from time import time
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import BayesianRidge, LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tools.eval_measures import rmse
from sklearn.inspection import permutation_importance
from sklearn import svm
from sklearn.svm import SVC
from pylab import rcParams
import dice_ml



import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/diabetic_data.csv', sep = ',')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

#### <font color='green'><i>Our data is pretty clean to start. There are some columms we will take care of below.</i></font>

<img src="https://www.signitysolutions.com/blog/wp-content/uploads/2020/09/How-AI-is-Transforming-Healthcare-1024x576.jpg">

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Understanding the Features</h1><hr>


Here we will examine the data and determine if any of the features are missing values or need to be coerced to a different data type. Some features can be dropped due to being grossly empty. Other may need to be cleaned up further. 

Input Variables:

1. <b>race</b> - <i>The race of the patient</i>
2. <b>gender</b> - <i>The gender of the patient (some marked as unknown)</i>
3. <b>age</b>	- <i>The age of the patient</i>
4. <b>admission_type_id</b> - <i>Integer identifier corresponding to 9 distinct values, for example, emergency, urgent, elective, newborn, and not available</i>
5. <b>discharge_disposition_id</b>	- <i>Integer identifier corresponding to 29 distinct values, for example, discharged to home, expired, and not available</i>
6. <b>admission_source_id</b> - <i>Integer identifier corresponding to 21 distinct values, for example, physician referral, emergency room, and transfer from a hospital</i>	
7. <b>time_in_hospital</b>	- <i>Integer number of days between admission and discharge</i>
8. <b>medical_specialty</b> - <i>Integer identifier of a specialty of the admitting physician, corresponding to 84 distinct values, for example, cardiology, internal medicine, family/general practice, and surgeon</i>
9. <b>num_lab_procedures</b>	- <i>Number of lab tests performed during the encounter</i>
10. <b>num_procedures</b>	- <i>4</i>
11. <b>num_medications</b>	- <i>Number of distinct generic names administered during the encounter</i>
12. <b>number_outpatient</b> -	<i>Number of outpatient visits of the patient in the year preceding the encounter</i>
13. <b>number_emergency</b>	- <i>Number of emergency visits of the patient in the year preceding the encounter</i>
14. <b>number_inpatient</b>	- <i>Number of inpatient visits of the patient in the year preceding the encounter</i>
15. <b>diag_1</b>	- <i>The primary diagnosis (coded as first three digits of ICD9); 848 distinct values</i>
16. <b>diag_2</b>	- <i>Secondary diagnosis (coded as first three digits of ICD9); 923 distinct values</i>
17. <b>diag_3</b>	- <i>Additional secondary diagnosis (coded as first three digits of ICD9); 954 distinct values</i>
18. <b>number_diagnoses</b>	- <i>Number of diagnoses entered to the system 0%</i>
19. <b>max_glu_serum</b>	- <i>Indicates the range of the result or if the test was not taken. Values: “>200,” “>300,” “normal,” and “none” if not measured</i>
20. <b>A1Cresult</b>	- <i>Indicates the range of the result or if the test was not taken. Values: “>8” if the result was greater than 8%, “>7” if the result was greater than 7% but less than 8%, “normal” if the result was less than 7%, and “none” if not measured.</i>
21. <b>metformin</b>	- <i>medical drug</i>
22. <b>repaglinid</b>e	- <i>medical drug</i>
23. <b>nateglinide</b>	- <i>medical drug</i>
24. <b>chlorpropamide</b>	- <i>medical drug</i>
25. <b>glimepiride</b>	- <i>medical drug</i>
26. <b>acetohexamide</b>	- <i>medical drug</i>
27. <b>glipizide</b>	- <i>medical drug</i>
28. <b>glyburide</b>	- <i>medical drug</i>
29. <b>tolbutamide</b>	- <i>medical drug</i>
30. <b>pioglitazone</b>	- <i>medical drug</i>
31. <b>rosiglitazone</b> - <i>medical drug	</i>
32. <b>acarbose</b>	- <i>medical drug</i>
33. <b>miglitol</b>	- <i>medical drug</i>
34. <b>troglitazone</b>	- <i>medical drug</i>
35. <b>tolazamide</b>	- <i>medical drug</i>
36. <b>examide</b>	- <i>medical drug</i>
37. <b>citoglipton</b>	- <i>medical drug</i>
38. <b>insulin</b>	- <i>medical drug</i>
39. <b>glyburide-metformin</b>	-<i> medical drug</i>
40. <b>glipizide-metformin</b>	- <i>medical drug</i>
41. <b>glimepiride-pioglitazone</b>	- <i>medical drug</i>
42. <b>metformin-rosiglitazone</b>	- <i>medical drug</i>
43. <b>metformin-pioglitazone</b>	- <i>medical drug</i>
44. <b>change</b>	- <i>Indicates if there was a change in diabetic medications (either dosage or generic name). Values: “change” and “no change”</i>
45. <b>diabetesMed</b>	-<i>Indicates if there was any diabetic medication prescribed. Values: “yes” and “no”</i>

Output variable (desired target):<br>
46. <b>readmitted</b> - <i>(<30 for less than 30 times, >30 for greater than 30 times, NO for not previously admitted) </i>

In [ ]:
#remove obvious irrelevant columns
df.drop(columns=['encounter_id','patient_nbr','weight', 'payer_code'], inplace=True)

In [ ]:
#find all columns that contain values of '?'. This can throw off our data. We will need to correct this.
sub_df = df.loc[: , (df == '?').any()]
print(sub_df.columns)

In [ ]:
#replace some values where it is marked as ? to unknown
df['race'] = df['race'].replace({'?':'Unknown'})
df['gender'] = df['gender'].replace({'?':'Unknown'})
df['medical_specialty'] = df['medical_specialty'].replace({'?':'Unknown'})
df['diag_1'] = df['diag_1'].replace({'?':'Unknown'})
df['diag_2'] = df['diag_2'].replace({'?':'Unknown'})
df['diag_3'] = df['diag_3'].replace({'?':'Unknown'})
#for the readmittance feature, lets replace none with Not Previously Admitted; less than 30 to Readmitted; and great than 30 to Readmitted
#we are waiving the 3 options down to 2. Our goal for this project is to simply get the usual readmitted patients to a point
#where they are not being readmitted. Thus, the binary classification of No Readmission vs Readmitted
df['readmitted'] = df['readmitted'].replace({'NO':'No Readmission'})
df['readmitted'] = df['readmitted'].replace({'<30':'Readmitted'})
df['readmitted'] = df['readmitted'].replace({'>30':'Readmitted'})
df['age'] = df['age'].replace({'[0-10)':5, 
                               '[10-20)':15, 
                               '[20-30)':25, 
                               '[30-40)':35, 
                               '[40-50)':45, 
                               '[50-60)':55 , 
                               '[60-70)':65 ,
                               '[70-80)':75,  
                               '[80-90)':85 ,
                               '[90-100)':95 })
#


In [ ]:
#verify changes
df.info()

In [ ]:
df.shape

In [ ]:
#Quick glance at distribution of data
df.hist(figsize=(20,15), bins=25)

In [ ]:
df.head()

In [ ]:
#pairplot for further visual observations
sns.pairplot(df)

In [ ]:
# Heat map to help show correlation matrix. Will allow aid in showing correlation between those numeric features
fig, ax = plt.subplots(figsize=(10,8)) 
sns.heatmap(df.corr(), annot=True)

<b>Observations</b>
+ While the overall correlation between features appear to be minimal, we'll reserve an in depth analysis for later. 
+ We're able to see the highest relation at 0.47: time in hospital vs. number of medications.
+ Upon brief observation, it appears that the higher number of medications the patient is on, the longer they stay in the hospital is.

<h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Further Visuals</h1><hr>

In [ ]:
#readmittance by race
fig, ax = plt.subplots(figsize=(15,5)) 
sns.countplot(data=df, x='race', hue='readmitted')
plt.title('Readmittance Occurence base on Race')
plt.legend(title='Readmittance', loc='upper right')

**Observations**
+ In our plot, we can see Caucasians have the highest amount of admittances, followed by African American, Asian, and then Hispanic
+ Some of our data did not include the race of the patient or it was not opted to not be disclosed by the patient.
<hr><br>

In [ ]:
#Readmittance by gender
px.histogram(df, x='readmitted', color='gender', title='Readmittance Occurence based on gender')

**Observations**
+ From our observation, we can see that females have higher rate of being readmitted (25.67k) vs males (21.23k)
+ The same is true for no readmittance
+ Perhaps there is some other clinical reason for this. We will not observe this in our project. 
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,5)) 
sns.countplot(data=df, x='age', hue='readmitted')
plt.title('Readmittance Occurence base on Age')
plt.legend(title='Readmittance', loc='upper right')

**Observations**
+ Of the age groups, our highest of admissions is betwen 70 and 80 yrs of age, followed by: 60-70, 80-90, then 50-60. It seems there is a much higher occurance in readmittance with elderly individuals.
+ The same seems to follow for those that are not readmitted.
<hr><br>

In [ ]:
dic_status = {1: "Emergency", 2: "Urgent", 3: "Elective", 4: "Newborn", 5: "Not Available", 6: "Not Noted", 7: "Trauma Center", 8: "Not Mapped"}
df["admission_type_id_mapped"] = df["admission_type_id"].map(dic_status)

fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='admission_type_id_mapped', hue='readmitted')
plt.title('Readmittance Occurence base on the Admission Type')
plt.legend(title='Readmittance', loc='upper right')
plt.xlabel("Reason For Being Admitted")
plt.tick_params(axis='x', rotation=30)

**Observations**
+ Our plot gives us insight into the reason being admitted. Patients generally come thru the emergency room, followed by (urgent) those of a direct admit by a referring physician, then those patients that request to be admitted.
<hr><br>

In [ ]:
dic_status = {1: " Physician Referral", 
              2: "Clinic Referral", 
              3: "HMO Referral", 
              4: "Transfer From Hospital", 
              5: "Transfer From SNF", 
              6: "Transfer From HCF", 
              7: "Emergency Room", 
              8: "Court/Law Enforcement", 
              9: "Not Available", 
              10: "Transfer From CAH",
              11: "Normal Delivery",
              12: "Premature Delivery",
              13: "Sick Baby",
              14: "Extramural Birth",
              15: "Not Available",
              17: "Not Available",
              18: "Transfer From HHA",
              19: "Readmission to Same HHA",
              20: "Not Mapped",
              21: "Unknown/Invalid",
              22: "Transfer From Hospital Inpatient",
              23: "Born Inside This Hospital",
              24: "Born Outside This Hospital",
              25: "Transfer From ASC",
              26: "Transfer From Hospice"}
df["discharge_disposition_id_mapped"] = df["admission_type_id"].map(dic_status)

fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='discharge_disposition_id_mapped', hue='readmitted')
plt.title('Readmittance Occurence base on the Original Admission Source')
plt.legend(title='Readmittance by Source', loc='upper right')
plt.xlabel("Admission Source")
plt.tick_params(axis='x', rotation=90)

**Observations**
+ Our highest class of those being readmitted are from direct Physician Referrals. In our project, we'll make efforts to keep the patient with their Physician and make recommendations on health items that can be utilized to keep the patient from being readmitted. We can use Counterfactual Explanations technique for this. 
+ Akin to the previous mentioned, Clinical Referrals and HMO Referrals are sending the most patients for Emergency Room referrals. We can make an impact on this where something health things can be better controlled. As mentioned, we will look at Counterfactual Explanations for guidance on what can be further managed by the patient and physician.
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='time_in_hospital', hue='readmitted')
plt.title('Time Spent In Hospital (in hours)')
plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Time Spent In Hospital (hours)")
plt.tick_params(axis='x', rotation=90)

**Observations**
+ It looks like the distribution yields that 3 hrs is the height of the curve for the time that a patient has spent in the hospital
+ Others have had stays as far as 14hrs when going to the emergency room! Wow!
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.barplot(data=df, x='num_lab_procedures', y='readmitted')
plt.title('Labs Taken Once Admitted')
#plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Labs Taken")

**Observations**
+ The amount of labs taken at once is higher for those being readmitted. Could this note that there are more test to be run?
+ Are these patients less healthier? Or do they have clinical issues that take further diagnosis?
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='num_procedures', hue='readmitted')
plt.title('Number of Previous Procedures Performed')
plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Previous Procedures")

**Observations**
+ Typically patients are not part of the group where they have had previous procedures. This is a good thing.
+ There can be health complications with diabetic patients where they have had previous operations.
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='num_medications', hue='readmitted')
plt.title('Number of Medications Patient Are On')
plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Medication Count")
plt.tick_params(axis='x', rotation=-90)

**Observations**
+ In my observation, the number of medications administered is alarmingly high! 
+ As we can see in the plot, some patients are reciging in the upwards of 60+ medications over the occurences of their stay at the hospitals
+ It appears that the larger set of individuals receive between 12 and 16 medications during their encounter
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='number_outpatient', hue='readmitted')
plt.title('Number of Previous Outpatient Visits, Previous Yr')
plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Previous Outpatient Visits, Previous Yr")
plt.tick_params(axis='x', rotation=90)

**Observations**
+ Typically, most non-admitted patients are not being seen in outpatient clinic throughout the previous year.
+ For those who were admitted, they too are typically not being seen in outpatient clinic through the previous year.
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='number_emergency', hue='readmitted')
plt.title('Number of Previous Emergency Room Visits, Previous Yr')
plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Previous Emergency Room Visits, Previous Yr")
plt.tick_params(axis='x', rotation=90)

**Observations**
+ Typically individuals did not have to go to the emergency room in the previous year.
<hr><br>

In [ ]:
fig, ax = plt.subplots(figsize=(15,7)) 
sns.countplot(data=df, x='number_inpatient', hue='readmitted')
plt.title('Number of Previous Inpatient Admissions, Previous Yr')
plt.legend(title='Times Readmitted', loc='upper right')
plt.xlabel("Previous Inpatient Admissions, Previous Yr")
plt.tick_params(axis='x', rotation=90)

**Observations**
+ As we look at inpatient visits, the proportionality vs outpatient begins to change. 
+ While those that are not readmitted remain highest, the number of those who were readmitted, are seeing higher inpatient admissions in the previous year.
<hr><br>

In [ ]:
px.histogram(df, x='max_glu_serum', color='readmitted', title='Maximum Glucose Serum', text_auto=True)

**Observations**
+ Most individuals did not have their glucose serum levels measured.
+ For those who did, most were normal
+ Where not normal, they were highly elevated! There appeared to be no 'slightly elevated' range. 
+ There appears to be room for health management here. Through proper glucuse management by the patient's physician, and medical devices, this can be brought under control!
<hr><br>

In [ ]:
px.histogram(df, x='A1Cresult', color='readmitted', title='A1C Result', text_auto=True)

**Observations**
+ Most patients did not have their A1C read via lab test.
+ Where the panel was taken it was largely high for the marjority of those patients! At 8 or above!
+ This is something that can be managed through proper care! We'll see later in Counterfactual Explanations where this change can make a difference in readmittance status.
<hr><br>

In [ ]:
px.histogram(df, x='metformin', color='readmitted', title='Drug: Metformin for High Blood Sugar', text_auto=True)

**Observations**
+ This plot was a measure of those who took Metforming HCL to control blood sugar.
+ Results see are for the measure of high blood sugar in a patient.
+ Most patients were not on metformin.
+ For those who are, their levels were steady.
+ This is a good sign! Most individuals that have high blood sugar, are keeping it under control. 
<hr><br>

In [ ]:
px.histogram(df, x='repaglinide', color='readmitted', title='Drug: Repaglinide for High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug
+ Most were not.
<hr><br>

In [ ]:
px.histogram(df, x='nateglinide', color='readmitted', title='Drug: Nateglinide for High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug
+ Most were not.
<hr><br>

In [ ]:
px.histogram(df, x='chlorpropamide', color='readmitted', title='Drug: Chlorpropamide for High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug
+ Most were not.
<hr><br>

In [ ]:
px.histogram(df, x='glimepiride', color='readmitted', title='Drug: glimepiride for High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug
+ Most were not.
<hr><br>

In [ ]:
px.histogram(df, x='acetohexamide', color='readmitted', title='Drug: acetohexamide for High Blood Sugar', text_auto=True)

**Observations**
+ Most patients were not on this drug.
<hr><br>


In [ ]:
px.histogram(df, x='glipizide', color='readmitted', title='Drug: glipizide for  High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug
+ Most were not.
<hr><br>

In [ ]:
px.histogram(df, x='glyburide', color='readmitted', title='Drug: glyburide for High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug
+ Most were not.
<hr><br>

In [ ]:
px.histogram(df, x='tolbutamide', color='readmitted', title='Drug: tolbutamide for  High Blood Sugar', text_auto=True)

**Observations**
+ Most patients were not on this drug.
<hr><br>

In [ ]:
px.histogram(df, x='pioglitazone', color='readmitted', title='Drug: pioglitazone for High Blood Sugar', text_auto=True)

**Observations**
+ Drug used in a small set of patients
+ Most were not on this drug.
<hr><br>

In [ ]:
px.histogram(df, x='rosiglitazone', color='readmitted', title='Drug: rosiglitazone for  High Blood Sugar', text_auto=True)

**Observations**
+ A small set of patients were on this drug.
+ Most were not on this drug.
<hr><br>

In [ ]:
px.histogram(df, x='acarbose', color='readmitted', title='Drug: acarbose for  High Blood Sugar', text_auto=True)

**Observations**
+ Most patients were not on this drug.
<hr><br>

In [ ]:
px.histogram(df, x='miglitol', color='readmitted', title='Drug: miglitol for High Blood Sugar', text_auto=True)

**Observations**
+ Most patients were not on this drug.
<hr><br>

In [ ]:
px.histogram(df, x='troglitazone', color='readmitted', title='Drug: troglitazone for High Blood Sugar', text_auto=True)

**Observations**
+ Most patients were not on this drug.
<hr><br>

In [ ]:
px.histogram(df, x='tolazamide', color='readmitted', title='Drug: tolazamide for High Blood Sugar', text_auto=True)

In [ ]:
px.histogram(df, x='examide', color='readmitted', title='Drug: Examide for Fluid Overload', text_auto=True)

**Observations**
+ No observations for this drug.
<hr><br>

In [ ]:
px.histogram(df, x='citoglipton', color='readmitted', title='Drug: Citoglipton for High Blood Sugar', text_auto=True)

**Observations**
+ No patient observations for this drug
<hr><br>

In [ ]:
px.histogram(df, x='insulin', color='readmitted', title='Drug: Insulin for Controlling Blood Sugar', text_auto=True)

**Observations**
+ Most patients did not have an issue with insulin levels.
+ For most, When checked, they were steady at the time of visit to the emergency room
+ For those that did not have steady levels they were largely down!
<hr><br>

In [ ]:
px.histogram(df, x='glyburide-metformin', color='readmitted', title='Drug: glyburide-metformin for', text_auto=True)

**Observations**
+ A small set of patients were on Glyburide-Metformin
+ Most patients were not on Glyburide-Metformin
<hr><br>

In [ ]:
px.histogram(df, x='glipizide-metformin', color='readmitted', title='Drug: glipizide-metformin for High Blood Sugar', text_auto=True)

**Observations**
+ Most patients were not on Glipizide-Metformin
<hr><br>

In [ ]:
px.histogram(df, x='glimepiride-pioglitazone', color='readmitted', title='Drug: glimepiride-pioglitazone for', text_auto=True)

**Observations**
+ Most patients were not on Glimepiride-Pioglitazone
<hr><br>

In [ ]:
px.histogram(df, x='metformin-rosiglitazone', color='readmitted', title='Drug: metformin-rosiglitazone for', text_auto=True)

**Observations**
+ Most patients were not on Metformin Rosiglitazone
<hr><br>

In [ ]:
px.histogram(df, x='metformin-pioglitazone', color='readmitted', title='Drug: metformin-pioglitazone for ', text_auto=True)

**Observations**
+ Most patients were not on Metformin-Pioglitazone
<hr><br>

In [ ]:
px.histogram(df, x='change', color='readmitted', title='Change Of Medications', text_auto=True)

**Observations**
+ This observation is for patients that had a change in their diabetic medications
+ For the majority, there was no change.
+ For those that did have a change, the larger portion of the patients where those who were not readmitted.
<hr><br>

In [ ]:
px.histogram(df, x='diabetesMed', color='readmitted', title='Number of Diabetes Medications', text_auto=True)

**Observations**
+ Most patients were already on diabetic medications. 
+ Of that group, most of those patients were not readmitted. 
<hr><br>

In [ ]:
#mapped columns. Done with visuals. we can drop those columns now. 
df.drop(columns=['admission_type_id_mapped', 'discharge_disposition_id_mapped'], inplace=True)

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Understanding the Task</h1>

***Identification of Business Goals***

+ Identify and compare the performance of the techniques(Logistic Regression, Decision Tree, Counterfactual Technique)
+ Search for and identify where success was best
+ Search for and Identify where improvements can be made
+ Make recommendations on what areas physicians should be focusing on for patient health management to help efforts toward decreasing diabetic patient readmissions to hospitals. 

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Engineering Features</h1><hr>

Now that basic business goals and objectives have been stated, we will build a basic model to get started. Before we can do this, we must work to encode the data where the values are not already numerical. I will prepare the features and target column for modeling with appropriate encoding and transformations. 

In [ ]:
df.info()

In [ ]:
#Just to get a general idea of where we stand on count for each of our values for target feature
df['readmitted'].value_counts()

In [ ]:
#Update gender to be numerical value
df['gender'] = df['gender'].replace({'Male':1, 'Female':2, 'Unknown/Invalid':3})

In [ ]:
#Identify and define the numerical and categorial columns
numerical_cols = [
    'age',
    'gender',
    'admission_type_id',
 'discharge_disposition_id',
 'admission_source_id',
 'time_in_hospital',
 'num_lab_procedures',
 'num_procedures',
 'num_medications',
 'number_outpatient',
 'number_emergency',
 'number_inpatient',
 'number_diagnoses'
]
categorical_cols = ['race',
                    'medical_specialty', 
                    'diag_1', 
                    'diag_2', 
                    'diag_3', 
                    'max_glu_serum', 
                    'A1Cresult', 
                    'metformin', 
                    'repaglinide', 
                    'nateglinide', 
                    'chlorpropamide', 
                    'glimepiride', 
                    'acetohexamide', 
                    'glipizide', 
                    'glyburide', 
                    'tolbutamide', 
                    'pioglitazone', 
                    'rosiglitazone', 
                    'acarbose', 
                    'miglitol', 
                    'troglitazone', 
                    'tolazamide', 
                    'examide', 
                    'citoglipton', 
                    'insulin', 
                    'glyburide-metformin', 
                    'glipizide-metformin', 
                    'glimepiride-pioglitazone', 
                    'metformin-rosiglitazone', 
                    'metformin-pioglitazone', 
                    'change', 
                    'diabetesMed', 
                    'readmitted']

In [ ]:
#Here we will use our Label Encoder to help digitize those columns which are not int/float
encoder_df = df.copy()
encoder = preprocessing.LabelEncoder()

def target_encoder(data):
    impute_ordinal = encoder.fit_transform(data)
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

for i in tqdm(range(len(categorical_cols))):
    target_encoder(encoder_df[categorical_cols[i]])

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Train/Test Split</h1>

With our data prepared, split it into a train and test set

In [ ]:
X = encoder_df.drop('readmitted', axis=1)
y = encoder_df['readmitted'].astype('int')
#print(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">A Baseline Model<h1><hr>

Before we build our first model, we want to establish a baseline.</p>

In [ ]:
models = ['Baseline','Logistic Regression', 'Decision Tree']
train_time = []
train_accuracy = []
test_accuracy = []
accuracy_score = []
AUC_score = []


dummy = DummyClassifier(strategy='uniform', random_state=42)
start_time = time()
dummy.fit(X_train, y_train)
train_time.append(time() - start_time)
train_accuracy.append(dummy.score(X_train, y_train))
test_accuracy.append(dummy.score(X_test, y_test))
accuracy_score.append('N/A')
AUC_score.append('N/A')

In [ ]:
print("#################BASELINE ANALYSIS####################\n")
print(f'Training time :{train_time}')
print(f'Training accuracy :{train_accuracy}')
print(f'Test accuracy :{test_accuracy}')
print(f'Accuracy score : Not Yet Available')
print(f'AUC score : Not Yet Available')
print("\n######################################################")

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">A Simple Model</h1><hr>
Here we will use Logistic Regression to build a basic model on our data. 

In [ ]:
maxit = 100000
lgr = LogisticRegression(solver='liblinear', random_state=42, max_iter=maxit)
start_time = time()
lgr.fit(X_train, y_train)
train_time.append(time() - start_time)
y_pred = lgr.predict(X_test)

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Score the Model</h1><hr>
Viewing our accuracy of this model

In [ ]:
train_accuracy.append(lgr.score(X_train, y_train))
test_accuracy.append(lgr.score(X_test, y_test))
accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
AUC_score.append(metrics.auc(fpr, tpr))

In [ ]:
print("#################LOGISTIC REGRESSION ANALYSIS####################\n")
print(f'Training time :{train_time[1]}')
print(f'Training accuracy :{train_accuracy[1]}')
print(f'Test accuracy :{test_accuracy[1]}')
print(f'Accuracy score : {accuracy_score[1]}')
print(f'AUC score : {AUC_score[1]}')
print("\n######################################################")

In [ ]:
cm_plot = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='icefire', fmt='d')
cm_plot.set_xlabel('Predictions')
cm_plot.set_ylabel('Actuals')
plt.title("Logistic Regression")

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Model Comparisons</h1><hr>

In [ ]:
from pandas.core.common import random_state
model_pipeline = []
model_pipeline.append(DecisionTreeClassifier(random_state=42))
cm_results = []

In [ ]:
model_pipeline

In [ ]:
for model in model_pipeline:
  start_time = time()
  model.fit(X_train, y_train)
  train_time.append(time() - start_time)
  y_pred = model.predict(X_test)
  train_accuracy.append(model.score(X_train, y_train))
  test_accuracy.append(model.score(X_test, y_test))
  accuracy_score.append(metrics.accuracy_score(y_test, y_pred))
  fpr, tpr, _thresholds = metrics.roc_curve(y_test, y_pred)
  AUC_score.append(metrics.auc(fpr, tpr))
  cm_results.append(confusion_matrix(y_test, y_pred))

In [ ]:
fig = plt.figure(figsize=(12, 10))
for i in range(len(cm_results)):
  cm = cm_results[i]
  model = models[i+2]
  sub = fig.add_subplot(2, 2, i+1).set_title(model)
  cm_plot = sns.heatmap(cm, annot=True, cmap='icefire', fmt='d')
  cm_plot.set_xlabel('Predictions')
  cm_plot.set_ylabel('Actuals')

In [ ]:
results1_df = pd.DataFrame({'Model': models, 'Train Time': train_time, 'Train Score': train_accuracy , 'Test Score': test_accuracy , 'Accuracy Score': accuracy_score, 'AUC Score': AUC_score})
results1_df

<br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Problem 11: Improving the Model</h1><hr>

In [ ]:
models = ['Logistic Regression', 'Decision Tree']
best_params = []
fit_time = []
acc_score = []
recall_score = []
prec_score = []
f1_score = []
r2_score = []
roc_auc_score = []

In [ ]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1-Lasso l2-Ridge
lgr = LogisticRegression(max_iter=maxit)
lgr_cv=GridSearchCV(lgr, grid, cv=10)
lgr_cv.fit(X_train, y_train)

In [ ]:
lgr_cv.best_params_

In [ ]:
lgr2=LogisticRegression(C=0.1, penalty="l2", random_state=42, max_iter=maxit) 
start_time = time()
lgr2.fit(X_train,y_train)
train_time = time() - start_time

In [ ]:
y_pred = lgr2.predict(X_test)

In [ ]:
cm_plot = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='icefire', fmt='d')
cm_plot.set_xlabel('Predictions')
cm_plot.set_ylabel('Actuals')
plt.title("Performance Confusion Matrix")

In [ ]:
best_params.append(lgr_cv.best_params_)
fit_time.append(train_time)
acc_score.append(metrics.accuracy_score(y_test, y_pred))
recall_score.append(metrics.recall_score(y_test, y_pred))
prec_score.append(metrics.precision_score(y_test, y_pred))
f1_score.append(metrics.f1_score(y_test, y_pred))
r2_score.append(metrics.r2_score(y_test, y_pred))
roc_auc_score.append(metrics.roc_auc_score(y_test, y_pred))

In [ ]:
importance = lgr2.coef_[0]
for i,v in enumerate(importance):
    print(f'Feature: {df.columns[i]} \t Score:{v}')

plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
encoder_df.info()

In [ ]:
params = {'max_depth': [1, 3, 5, 7, 11, 13, 15, 17, 19, 21, 23],
         'min_samples_split': [1, 50, 2],
          'criterion': ['gini', 'entropy'],
          'min_samples_leaf': [1]}
dtree = DecisionTreeClassifier()
dtree_cv = GridSearchCV(dtree, params, cv=10)
dtree_cv.fit(X_train, y_train)

In [ ]:
dtree_cv.best_params_

In [ ]:
dtree2=DecisionTreeClassifier(criterion='gini', max_depth=7, min_samples_leaf=1, min_samples_split=50, random_state=42)
start_time = time()
dtree2.fit(X_train,y_train)
train_time = time() - start_time

In [ ]:
y_pred = dtree2.predict(X_test)

In [ ]:
cm_plot = sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap='icefire', fmt='d')
cm_plot.set_xlabel('Predictions')
cm_plot.set_ylabel('Actuals')
plt.title("Performance Confusion Matrix")

In [ ]:
best_params.append(dtree_cv.best_params_)
fit_time.append(train_time)
acc_score.append(metrics.accuracy_score(y_test, y_pred))
recall_score.append(metrics.recall_score(y_test, y_pred))
prec_score.append(metrics.precision_score(y_test, y_pred))
f1_score.append(metrics.f1_score(y_test, y_pred))
r2_score.append(metrics.r2_score(y_test, y_pred))
roc_auc_score.append(metrics.roc_auc_score(y_test, y_pred))

In [ ]:
dtree_coeffs = pd.Series(dtree2.feature_importances_, index=X_train.columns)
dtree_coeffs.plot(kind='bar', figsize=(15,10))
plt.title('Decision Tree Coefficients')
plt.xlabel('X_train Values')
plt.tick_params(axis='x', rotation=-80)

In [ ]:
#top 15 features, according to decision tree
dtree_coeffs.sort_values(ascending=False).head(15)

In [ ]:
results2_df = pd.DataFrame({'Model': models, 'Best_Params': best_params, 'Train_Time': fit_time, 'Accuracy_Score': acc_score, 'Recall_Score': recall_score, 'Precision_Score': prec_score, 'F1_Score': f1_score, 'R2_Score': r2_score, 'ROC_AUC_Score': roc_auc_score})
results2_df

<hr><br><br><h1 align='center' style="color:#643872; font-family:Merriweather, sans-serif; font-size:36px;">Implementation of Counterfactual Explanations</h1><hr><br><img src="images/african-american-doctor-discussing-healthcare-trea-2021-12-09-02-41-56-utc.jpg" width="90%" height="90%">

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
X = encoder_df.drop('readmitted', axis=1)
y = encoder_df['readmitted'].astype('int')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123, stratify = y)

In [ ]:
encoder_df.columns

In [ ]:
#array of features that a physician can actually influence in an attempt to keep a diabetic
# patient healthier. some features such as race, age, gender, etc. cannot be modified to change health outcome. 
# Thus, we omit from our continuous feature array. We only want items that can be manipulated by medication, nutrition,
# excercise, and/or some other means deemed acceptable by physicians/researchers. 
cf_array = ['num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed']

In [ ]:
# dice complained about not readily recognizing the int values. encoder_df.info() 
# showed them as objects. so, manually converting over to int type.
encoder_df['diag_1'] = encoder_df['diag_1'].astype('float')
encoder_df['diag_2'] = encoder_df['diag_1'].astype('float')
encoder_df['diag_3'] = encoder_df['diag_1'].astype('float')
encoder_df['max_glu_serum'] = encoder_df['max_glu_serum'].astype('int')
encoder_df['A1Cresult'] = encoder_df['A1Cresult'].astype('int')
encoder_df['max_glu_serum'] = encoder_df['max_glu_serum'].astype('int')
encoder_df['metformin'] = encoder_df['max_glu_serum'].astype('int')
encoder_df['repaglinide'] = encoder_df['repaglinide'].astype('int')
encoder_df['nateglinide'] = encoder_df['nateglinide'].astype('int')
encoder_df['chlorpropamide'] = encoder_df['chlorpropamide'].astype('int')
encoder_df['glimepiride'] = encoder_df['glimepiride'].astype('int')
encoder_df['acetohexamide'] = encoder_df['acetohexamide'].astype('int')
encoder_df['glipizide'] = encoder_df['glipizide'].astype('int')
encoder_df['glyburide'] = encoder_df['glyburide'].astype('int')
encoder_df['tolbutamide'] = encoder_df['tolbutamide'].astype('int')
encoder_df['pioglitazone'] = encoder_df['pioglitazone'].astype('int')
encoder_df['rosiglitazone'] = encoder_df['rosiglitazone'].astype('int')
encoder_df['acarbose'] = encoder_df['acarbose'].astype('int')
encoder_df['miglitol'] = encoder_df['miglitol'].astype('int')
encoder_df['troglitazone'] = encoder_df['troglitazone'].astype('int')
encoder_df['tolazamide'] = encoder_df['tolazamide'].astype('int')
encoder_df['examide'] = encoder_df['examide'].astype('int')
encoder_df['citoglipton'] = encoder_df['citoglipton'].astype('int')
encoder_df['insulin'] = encoder_df['insulin'].astype('int')
encoder_df['glyburide-metformin'] = encoder_df['glyburide-metformin'].astype('int')
encoder_df['glipizide-metformin'] = encoder_df['glipizide-metformin'].astype('int')
encoder_df['glimepiride-pioglitazone'] = encoder_df['glimepiride-pioglitazone'].astype('int')
encoder_df['metformin-rosiglitazone'] = encoder_df['metformin-rosiglitazone'].astype('int')
encoder_df['metformin-pioglitazone'] = encoder_df['metformin-pioglitazone'].astype('int')
encoder_df['change'] = encoder_df['change'].astype('int')
encoder_df['diabetesMed'] = encoder_df['diabetesMed'].astype('int')


In [ ]:
# step 1 - dice_ml.Data
d = dice_ml.Data(dataframe = encoder_df, 
             continuous_features = cf_array, outcome_name = 'readmitted')

In [ ]:
d

**Diverse Counterfactual Explanations (DiCE) for ML: https://github.com/interpretml/DiCE**

In [ ]:
dtclf = DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
# use the sklearn backend
m = dice_ml.Model(model = dtclf, backend = "sklearn")

In [ ]:
exp = dice_ml.Dice(d, m, method = 'random')

In [ ]:
e1 = exp.generate_counterfactuals(X_test[0:1],
                             total_CFs = 2,
                             desired_class = "opposite")

e1.visualize_as_dataframe()

In [ ]:
e2 = exp.generate_counterfactuals(X_test[0:1],
                             total_CFs = 2,
                             desired_class = "opposite",
                             features_to_vary = cf_array)

e2.visualize_as_dataframe()